# Part 4: Accessing other Things, actually the instructor's things!


Now we are going control things in other AWS IoT Core account and other MQTT broker.

### Let's recover some data

In [1]:
#Hour and minute when this started to create unique names
%store -r HM
#Home directory of the user running this notebook
%store -r home


### Now we are going to download a pre-generated certificate and access keys to the target account

In [2]:

! mkdir -p "{home}/aws_iot_hub"

other_certificate_file = "{}/aws_iot_hub/certificate-hub.pem".format(home)
other_public_key = "{}/aws_iot_hub/public-key-hub.pem".format(home)
other_private_key = "{}/aws_iot_hub/private-key-hub.pem".format(home)
other_ca_file = "{}/aws_iot_hub/ca.pem".format(home)

%store other_certificate_file
%store other_public_key
%store other_private_key
%store other_ca_file


Stored 'other_certificate_file' (str)
Stored 'other_public_key' (str)
Stored 'other_private_key' (str)
Stored 'other_ca_file' (str)


In [3]:
! wget https://s3.amazonaws.com/healthcare-workshop/certificate-hub.pem -O {other_certificate_file}
! wget https://s3.amazonaws.com/healthcare-workshop/public-key-hub.pem -O {other_public_key}
! wget https://s3.amazonaws.com/healthcare-workshop/private-key-hub.pem -O {other_private_key}
! wget https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem -O {other_ca_file}


--2018-05-22 15:10:42--  https://s3.amazonaws.com/healthcare-workshop/certificate-hub.pem
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.129.29
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.129.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1220 (1.2K) [application/x-x509-ca-cert]
Saving to: ‘/home/ec2-user/aws_iot_hub/certificate-hub.pem’

/home/ec2-user/aws_ 100%[===================>]   1.19K  --.-KB/s    in 0s      

2018-05-22 15:10:42 (94.1 MB/s) - ‘/home/ec2-user/aws_iot_hub/certificate-hub.pem’ saved [1220/1220]

--2018-05-22 15:10:42--  https://s3.amazonaws.com/healthcare-workshop/public-key-hub.pem
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.1.147
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.1.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451 [application/x-x509-ca-cert]
Saving to: ‘/home/ec2-user/aws_iot_hub/public-key-hub.pem’

/home/ec2-user/aws_ 100%[==================

Double check that you have the 4 files required for TLS mutual authentication used by AWS IoT
1. Private Key
1. Public Key
1. Certificate
1. Root CA

In [5]:
! printf "Private Key {other_private_key} " && [ -f {other_private_key} ] && printf "found" || printf "not found"
! printf "\nPublic Key {other_public_key} " && [ -f {other_public_key} ] && printf "found" || printf "not found"
! printf "\nCertificate {other_certificate_file} " && [ -f {other_certificate_file} ] && printf "found" || printf "not found"
! printf "\nCertification Authority {other_ca_file} " && [ -f {other_ca_file} ] && printf "found" || printf "not found"

Private Key /home/ec2-user/aws_iot_hub/private-key-hub.pem found
Public Key /home/ec2-user/aws_iot_hub/public-key-hub.pem found
Certificate /home/ec2-user/aws_iot_hub/certificate-hub.pem found
Certification Authority /home/ec2-user/aws_iot_hub/ca.pem found

# Connecting to our AWS IoT Account

AWS IoT provides account-specific endpoints for your things to connect. We are using the instructor workshop endpoint at this time:

In [62]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

other_endpoint_address = "a2enzbgohiblz2.iot.us-east-1.amazonaws.com"
%store other_endpoint_address

client_id="NotebookCient{}".format(HM)
other_mqtt = AWSIoTMQTTClient(client_id)
other_mqtt.configureEndpoint(other_endpoint_address, 8883)
other_mqtt.configureCredentials(other_ca_file, other_private_key, other_certificate_file)
other_mqtt.configureConnectDisconnectTimeout(10)
other_mqtt.connect()
other_mqtt

Stored 'other_endpoint_address' (str)


# Let's control our lamp

## Turn on lamp

In [63]:
other_topic_name ="/request"
publish_qos = 0
msg = "{ gpio: { pin: 25, state: 0}}"
other_mqtt.publish(other_topic_name, msg, publish_qos)


False

## Turn off lamp

In [64]:
other_topic_name ="/request"
publish_qos = 0
msg = "{ gpio: { pin: 25, state: 1}}"
other_mqtt.publish(other_topic_name, msg, publish_qos)


True

# Let's subscribe to IoT Button

In [65]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id="NotebookCient_2{}".format(HM)
other_mqtt = AWSIoTMQTTClient(client_id)
other_mqtt.configureEndpoint(other_endpoint_address, 8883)
other_mqtt.configureCredentials(other_ca_file, other_private_key, other_certificate_file)
other_mqtt.configureConnectDisconnectTimeout(10)
other_mqtt.connect()
other_mqtt
button_topic_name ="healthcare/button"

subscribe_qos = 1

def print_message(client, userdata, message):
    print("\nReceived a new message: \n")
    print(message.payload.decode("utf-8") )
    print("\n--------------\n")
    
other_mqtt.subscribe(button_topic_name, subscribe_qos, print_message)

True


Received a new message: 

{"serialNumber": "G030JF05639203PU", "batteryVoltage": "1629mV", "clickType": "SINGLE"}

--------------



All working fine, done for now, disconnect.

In [66]:
other_mqtt.disconnect()


True

Now we don't want to loose this data, right?
Let's proceed to the [Storing Telemetry Data](aws-iot-storing-telemetry.ipynb) notebook right now!